In [ ]:
import sys,os,re
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import seaborn           as sb

from theory_code.distance_theory import TheoryCalcs
from mock_data_code import MockCalcs

from itertools import product
from copy      import deepcopy
from time      import time

from scipy.interpolate import interp1d
from scipy.integrate   import trapz

import warnings
warnings.filterwarnings('ignore')

import matplotlib
from matplotlib import rc
from matplotlib.pyplot import cm
from matplotlib.colors import LogNorm

rc('text', usetex=True)
rc('font', family='serif')
matplotlib.rcParams.update({'font.size': 18})


sidelegend = {'bbox_to_anchor': (1.04,0.5), 
              'loc': "center left",
              'frameon': False}

In [ ]:
fiducial = {'H0': 68.53,
            'omegam': 0.2948,
            'ombh2': 0.02218,
            'omk': 0.,
            'epsilon0_EM': 0.1,
            'epsilon0_GW': -0.1,
            'mnu': 0.06,
            'nnu': 3.,
            'MB': -19.2435}
settings = {'zmin': 0.001,
            'zmax': 5.,
            'Nz': 1000,
            'zdrag': 1060,
            'DDR_model': 'constant'}


In [ ]:
obs_settings = {'BAO' : {'distribution': 'binned',
                        'error_type' : 'percentage',
                        'correlation': False,
                        'BAO_file_path' : './mock_data/',
                        'zmin': 0.65,
                        'zmax': 1.85,
                        'N_bin': 14},
                'SN' :{'distribution': 'Euclid',
                        'error_type' : 'percentage',
                        'correlation': False,
                        'SN_file_path' : './mock_data/',
                        'zmin': 0.1,
                        'zmax': 1.0,
                        'N_SN': 8800},
               'GW' :{'distribution': 'BNS',
                      'merger_rate' : 'BNS',
                        'error_type' : 'observational_error',
                        'GW_file_path' : './mock_data/',
                        'correlation': False,
                        'zmin': 0.001,
                        'zmax': 5.0,
                        'N_gw': 1000}}

In [ ]:
#THEORY

theory = TheoryCalcs(settings,fiducial)
z_camb  = np.linspace(0.001,5.,10000)
dA      = theory.dA(z_camb)
Hz      = theory.Hz(z_camb)*3*1e5
dL_EM_theo = theory.DL_EM(z_camb)
dL_GW_theo = theory.DL_GW(z_camb)
mB_theo = 5 * np.log10(theory.DL_EM(z_camb)) + 25 + fiducial['MB']
dL_0       = (1+z_camb)**(2)*dA

In [ ]:
#MOCK
mock = MockCalcs(settings, obs_settings, fiducial, theory)

if 'BAO' in obs_settings:
    BAO_data = mock.data_BAO
if 'SN' in obs_settings:
    SN_data = mock.data_SN
if 'GW' in obs_settings:    
    GW_data = mock.data_GW

In [ ]:
if 'BAO' in obs_settings:
    dA_data =  ((BAO_data['DV']*theory.rdrag)**3 / (BAO_data['z'] * (1 + BAO_data['z'])**2 * BAO_data['DH']*theory.rdrag))**0.5
    dA_error = 0.05*dA_data
    
    plt.figure(figsize=(8, 6))
    plt.plot(z_camb, dA, label="Theoretical dA", color='red', linewidth=2)
    plt.errorbar(BAO_data['z'], dA_data, yerr=dA_error, fmt='o',label='dA mock')
    plt.xlim(0.3, 2.0)
    plt.ylim(1300, 2100)
    plt.xlabel("Redshift (z)")
    plt.ylabel("DA")
    plt.title("DA vs Redshift")
    plt.legend()
    plt.show()

In [ ]:
if 'BAO' in obs_settings:
    Hz_data = 3*1e5/( BAO_data['DH']*theory.rdrag)
    Hz_error = 0.05*Hz_data
    
    plt.figure(figsize=(8, 6))
    plt.plot(z_camb, Hz, label="Theoretical Hz", color='red', linewidth=2)
    plt.errorbar(BAO_data['z'], Hz_data, yerr=Hz_error, fmt='o',label='Hz mock')
    plt.xlim(0.3, 2.0)
    plt.ylim(50, 250)
    plt.xlabel("Redshift (z)")
    plt.ylabel("Hz")
    plt.title("Hz vs Redshift")
    plt.legend()
    plt.show()


In [ ]:
if 'SN' in obs_settings:
    plt.figure(figsize=(8, 6))
    plt.plot(z_camb, mB_theo, label="Theoretical mB", color='red', linewidth=2, zorder=2)
    plt.errorbar(SN_data['z'], SN_data['mB'], yerr=SN_data['err_mB'], label="mB data", 
                 capsize=2, markersize=5, alpha=0.8,  zorder=1)
    plt.xlim(0.1,1.0)
    plt.xlabel("Redshift (z)")
    plt.ylabel("mB")
    plt.title("mB vs Redshift")
    plt.legend()
    plt.show()

In [ ]:
if 'SN' in obs_settings:
    N = 100
    random_indices = np.random.choice(len(SN_data['z']), N, replace=False)
    
    
    z_SN_subset = SN_data['z'][random_indices]
    dL_EM_subset = theory.DL_EM(SN_data['z'])[random_indices]
    
    
    sigma_dL_subset = 0.05 * dL_EM_subset
    
    dL_EM_shifted =  np.random.normal(dL_EM_subset, sigma_dL_subset)
    
    plt.figure(figsize=(8, 6))
    plt.errorbar(z_SN_subset, dL_EM_shifted, yerr=sigma_dL_subset, color='darkslategrey',
                 fmt='o', ecolor='green', capsize=2, markersize=2, alpha=0.8,  zorder=1, label='Observed dL (EM)')
    plt.plot(z_camb, dL_EM_theo, label="Theoretical dL", color='blue',ls = '--', linewidth=3, zorder=2)
    plt.plot(z_camb, dL_0, label="Theoretical dL with $\epsilon=0$", color='red', linewidth=3, zorder=3, ls= '--')
    plt.xlim(0.1, 1.0)
    plt.ylim(0.1, 10000)
    plt.xlabel('Redshift (z)')
    plt.ylabel('Luminosity Distance $d_L$ (Mpc)')
    plt.title('$d_L$  vs. z $\epsilon_{EM}=0.1$')
    plt.legend()
    plt.show()

In [ ]:
if 'GW' in obs_settings:
    plt.figure(figsize=(8, 6))
    
    
    plt.errorbar(GW_data['z'], GW_data['dL'], yerr=GW_data['err_dL'], 
                 fmt='o', color='orchid',ecolor='gray', capsize=2, markersize=2, alpha=0.8,  zorder=1, label='Observed dL (GW)')
    
    plt.plot(z_camb, dL_GW_theo, label="Theoretical dL (GW)", color='red', linewidth=3, zorder=2)
    plt.plot(z_camb, dL_0, label="Theoretical dL with $\epsilon=0$", color='green', linewidth=3, zorder=3, ls= '--')
    
    plt.xlabel('Redshift (z)')
    plt.ylabel('Luminosity Distance $d_L$ (Mpc)')
    plt.title('$d_L$  vs. z $\epsilon_{GW}=-0.1$')
    plt.legend()
    plt.show()

In [ ]:
if 'SN' in obs_settings and 'GW' in obs_settings:
    plt.figure(figsize=(8, 6))
    
    
    plt.errorbar(GW_data['z'], GW_data['dL'], yerr=GW_data['err_dL'], 
                 fmt='o', color='orchid',ecolor='gray', capsize=2, markersize=2, alpha=0.8,  zorder=1, label='Observed dL (GW)')
    
    plt.plot(z_camb, dL_GW_theo, label="Theoretical dL (GW)", color='red', linewidth=3, zorder=3)
    plt.errorbar(z_SN_subset, dL_EM_shifted, yerr=sigma_dL_subset, color='darkslategrey',
                 fmt='o', ecolor='green', capsize=2, markersize=2, alpha=0.8,  zorder=4, label='Observed dL (EM)')
    plt.plot(z_camb, dL_EM_theo, label="Theoretical dL", color='blue',ls = '--', linewidth=3, zorder=2)
    plt.xlabel('Redshift (z)')
    plt.ylabel('Luminosity Distance $d_L$ (Mpc)')
    plt.title('$d_L$  vs. z $\epsilon_{EM}=0.1$, $\epsilon_{GW}=-0.1$')
    plt.legend()
    plt.show()

In [ ]:
if 'SN' in obs_settings and 'GW' in obs_settings:
    plt.figure(figsize=(8, 6))
    
    plt.errorbar(GW_data['z'], GW_data['dL'], yerr=GW_data['err_dL'], 
                 fmt='o',color='orchid', ecolor='gray', capsize=2, markersize=3, alpha=0.8,  zorder=1, label='Observed dL (GW)')
    
    plt.plot(z_camb, dL_GW_theo, label="Theoretical dL (GW)", color='red', linewidth=3, zorder=3)
    plt.errorbar(z_SN_subset, dL_EM_shifted, yerr=sigma_dL_subset, color='darkslategrey',
                 fmt='o', ecolor='green', capsize=2, markersize=3, alpha=0.8,  zorder=4, label='Observed dL (EM)')
    plt.plot(z_camb, dL_EM_theo, label="Theoretical dL", color='blue',ls = '--', linewidth=3, zorder=2)
    plt.xlim(0.1, 1.0)
    
    plt.ylim(0.1, 10000)
    plt.xlabel('Redshift (z)')
    plt.ylabel('Luminosity Distance $d_L$ (Mpc)')
    plt.title('$d_L$  vs. z $\epsilon_{EM}=0.1$, $\epsilon_{GW}=-0.1$')
    plt.legend()